In [ ]:
from playwright.async_api import async_playwright
import os
import json


## Define Variables


In [ ]:
LOGIN_PAGE_URL = "https://www.beenverified.com/app/login"
HOME_PAGE_URL = "https://www.beenverified.com/rf/dashboard"
REPORT_PAGE_URL = "https://www.beenverified.com/api/v5/reports"
SEARCH_PAGE_URL = "https://www.beenverified.com/rf/search/person"

In [ ]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)



In [ ]:
context = await browser.new_context(storage_state="playwright/.auth/state.json")
page =  await context.new_page()
await page.goto(HOME_PAGE_URL)


## Login

In [ ]:
url  = "https://www.beenverified.com/rf/search/person?age=&city=&fname=Samama&ln=Mahmud&mn=&state=All"
# url = "https://www.beenverified.com/rf/search/person?age=&city=&fname=BRETT&ln=cox&mn=&state="
await page.goto(url)

In [ ]:
container_results = await page.query_selector(".css-ts1zsd")
container_results

In [ ]:
#get the user container 
container_users = await container_results.query_selector_all(".css-1mvdt3q")
container_users

In [ ]:
container_user = container_users[0]
container_user

In [ ]:
txt = await container_user.text_content()

In [ ]:
'Relatives' in txt

In [ ]:
#go to the report page
view_report = await container_user.query_selector(".css-gnnc3j")

In [ ]:
view_report

In [ ]:
# await view_report.click()

In [ ]:
async with page.expect_popup() as popup_info:
    view_report = await container_user.query_selector(".css-gnnc3j")
    await view_report.click()# Replace with the actual button selector
    new_page = await popup_info.value

# get top 5 users

In [ ]:
async def open_report_page(page, container_user):
    async with page.expect_popup() as popup_info:
        view_report = await container_user.query_selector(".css-gnnc3j")
        await view_report.click()# Replace with the actual button selector
        new_page = await popup_info.value
        return new_page


In [ ]:
new_page = await open_report_page(page, container_user)
new_page.url

# Exetra information of the user

In [ ]:
async def extract_phone_numbers(page):
    container_phone = await new_page.query_selector("#phone-numbers-section")
    phone_numbers = await container_phone.query_selector_all(".css-1vugsqn")
    phone_numbers = [await phone_number.text_content() for phone_number in phone_numbers]
    return phone_numbers

await extract_phone_numbers(new_page)


In [ ]:
async def extract_addresses(page):
    addresses_list = []
    container_address = await page.query_selector("#address-history-section")
    addresses = await container_address.query_selector_all(".css-1q4wjho")
    for address in addresses:
        address_fields = await address.query_selector_all(".css-zv7ju9")
        addresses_txt = [await address_field.text_content() for address_field in address_fields]
        addresses_list.append(' '.join(addresses_txt))
        
    return addresses_list

await extract_addresses(new_page)


In [ ]:
async def extract_email_list(page):
    email_container = await page.query_selector("#email-section")
    email_elements = await email_container.query_selector_all(".css-1vugsqn")
    email_list = [await email_element.text_content() for email_element in email_elements]
    return email_list

await extract_email_list(new_page)


In [ ]:
async def get_lead_info(new_page):
    extra_phone_numbers = await extract_phone_numbers(new_page)  # noqa: F821
    extra_addresses = await extract_addresses(new_page)
    extra_emails = await extract_email_list(new_page)
    return extra_phone_numbers, extra_addresses, extra_emails

extra_phone_numbers, extra_addresses, extra_emails = await get_lead_info(new_page)
extra_phone_numbers, extra_addresses, extra_emails

    
    
    

In [ ]:
#phone number
container_phone = await new_page.query_selector("#phone-numbers-section")
container_phone

In [ ]:
#get the phone number .css-1vugsqn
phone_numbers = await container_phone.query_selector_all(".css-1vugsqn")
phone_numbers

In [ ]:
for phone in phone_numbers:
    print(await phone.text_content())


In [ ]:
#email #email-section
container_email = await new_page.query_selector("#email-section")

In [ ]:
# get the email .css-1vugsqn
emails = await container_email.query_selector_all(".css-1vugsqn")
emails

In [ ]:
for email in emails:
    print(await email.text_content())

In [ ]:
#address
container_address = await new_page.query_selector("#address-history-section")
container_address

In [ ]:
addresses_cells = await container_address.query_selector_all(".css-1q4wjho")

In [ ]:
addresse_cell = addresses_cells[0]
addresse_cell

In [ ]:
adress = await addresse_cell.query_selector_all(".css-zv7ju9")

In [ ]:
for cell in adress:
    print(await cell.text_content())

In [ ]:
for addresse_cell in addresses_cells:
    addresse = await addresse_cell.query_selector(".css-zv7ju9")
    print(await addresse.text_content())

In [ ]:
await addresse.text_content()

In [ ]:
await new_page.close()

In [ ]:
await browser.close()
await pw.stop()